In [3]:
""" It is recommmended NOT to change codes in this cell """
try:
    import google.colab
    !pip install bitsandbytes==0.43.0
    !pip install datasets==2.10.1
    !pip install transformers==4.38.2
    !pip install peft==0.11.0
    !pip install sentencepiece==0.1.99
    !pip install -U accelerate==0.28.0
    !pip install colorama==0.4.6
    !pip install git+https://github.com/bhbbbbb/f-final
except:
    pass

ho colab


In [1]:
import warnings
warnings.filterwarnings("ignore")
from transformers import GenerationConfig
import final
from final.llm import LLM
final.data.set_password(...)

## Instruction & Inference Dataset

It's recommended not to modify this block, but one can play around and check what is used in the inference_dataset.


In [ ]:
N_TAGS_PER_PRODUCT = 7
INSTRUCTION = (
    'Predict the next product tags based on browsing histories. '
    f'Provide exactly {N_TAGS_PER_PRODUCT} output tags, comma-separated, sorted by relevance. '
)
PRODUCT_RAW_TAGS = final.data.product_tags_v5_en()

inference_dataset = final.InferenceDataset(
    raw_tags=PRODUCT_RAW_TAGS,
    # user_names=final.data.default_user_names(),
    n_tags_per_product=N_TAGS_PER_PRODUCT,
    legacy_output_first_time_buyer=False,
)
print(PRODUCT_RAW_TAGS)
print(INSTRUCTION)
print(inference_dataset[0])
print(inference_dataset[1])
print(inference_dataset[3])
print(inference_dataset[5])

## LLM Preparation
Download and initialize the corresponding LLM and checkpoint (Both need to be installed).

In [ ]:
# model_name = "/content/taide_7b"
# !wget -O taide_7b.zip "https://www.dropbox.com/scl/fi/rlzdoidgejt9wiox88gz5/taide_7b.zip?rlkey=vtuc8xq7mq2d0dpuijxlb3jfj&dl=0"
# # !gdown 1OiO5N9-wXDPX9XuGnPP-cgOoyFy4p3FD # 如果上面的連結失效（無法下載模型），可以將這行最前面的 "#" 刪除，即可正常下載
# !unzip taide_7b.zip

In [ ]:
ckpt_name = '/content/ckpt-14'
!wget -O "ckpt-llama2-14.zip" "https://www.dropbox.com/scl/fi/i9xfimsebp8i9b07013iu/ckpt-llama2-14.zip?rlkey=schxa8tln8oa55kklobeuxgd0&st=dqhm9r4r&dl=0"
!unzip "ckpt-llama2-14.zip"

In [3]:
model_name = 'togethercomputer/Llama-2-7B-32K-Instruct'
llm = LLM(
    model_name=model_name,
    instruction=INSTRUCTION,
    ckpt_name=ckpt_name,
    real_ckpt_dir='ckpts'
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.27s/it]


loading checkpoint from ckpts/ckpt-14


## Generation

There are three available generation methods.
It's recommended to try single_inference and multi_beam_inference first.

### single_inference

Just inference the model once.

### iterative_inference.

Need to specify `n` (`n >= 2`). This method used the output of $i$-th iteration as the input of $i+1$-th iteration.

### multi_beam_inference

Similar to single inference, but use multiple generation configs. 


In [ ]:
"""Single Inference"""

max_len = 128 # not sure
generation_config = GenerationConfig(
    do_sample=True,
    temperature=0.1, # important
    num_beams=1,
    top_p=0.5, # not sure
    no_repeat_ngram_size=3, # important (higher less diversity)
    pad_token_id=2,
)
single_inference_results = {}
for i, pid in enumerate(final.AVAILABLE_PRODUCT_IDS):
    d = inference_dataset[i]
    predict = llm.single_inference(d, generation_config, max_len, verbose=False)
    inputs_ = '\n'.join(d['input'].split('\n')[:-1])
    print(f'inputs{i}:\n{d["instruction"]}\n{inputs_}。\n')
    print(f'outputs:\n' + predict)
    print('-' * 50)
    single_inference_results[pid] = predict

In [ ]:
"""MultiBeam Inference"""
max_len = 128 # not sure
generation_configs = [
    GenerationConfig(
        do_sample=True,
        temperature=0.1, # important
        num_beams=1,
        top_p=0.3, # not sure
        no_repeat_ngram_size=1, # important (higher less diversity)
        pad_token_id=2,
    ),
    GenerationConfig(
        do_sample=True,
        temperature=0.1, # important
        num_beams=1,
        top_p=0.3, # not sure
        no_repeat_ngram_size=3, # important (higher less diversity)
        pad_token_id=2,
    ),
    GenerationConfig(
        do_sample=True,
        temperature=0.1, # important
        num_beams=1,
        top_p=0.3, # not sure
        no_repeat_ngram_size=5, # important (higher less diversity)
        pad_token_id=2,
    ),
]
multi_beam_inference_results = {}
for i, pid in enumerate(final.AVAILABLE_PRODUCT_IDS):
    d = inference_dataset[i]
    predicts = llm.multi_beam_inference(d, generation_configs, max_len, verbose=False)
    inputs_ = '\n'.join(d['input'].split('\n')[:-1])
    print(f'inputs:\n{d["instruction"]}\n{inputs_}。\n')
    print(f'outputs:\n2. ' + predicts)
    print('-' * 50)
    multi_beam_inference_results[pid] = predicts

## Results Parsing

Since it's hard for LLM to perfectly follow the given format, the generation may need to manually parsed.

In [2]:
import final
print(final.data.product_tags_v5_en())

{5: ['Love Ataulfo Mango', 'Mango', 'Mango Ice', 'Fresh Mangoes', 'Exquisite Fruits', 'Fruit Gift Box', 'Juicy Mango', 'Premium Mango', 'Summer Delight', 'Perfect Gift'], 23: ['Frozen Broccoli', 'Cruciferous Vegetables', 'Cancer-fighting', 'Low-Calorie Diet', 'Healthy Eating', 'Quick Meal Prep', 'Fitness Recipes', 'Weight Loss', 'Low-Carb Option', 'Salad Ingredient'], 24: ['Frozen Cauliflower', 'Healthy Eating', 'Meal Preparation', 'Fitness', 'Weight Control', 'Quick Meals', 'Frozen Vegetables', 'Nutritious Ingredients', 'Diet Control', "Kids' Lunch"], 25: ['Healthy Eating', 'Frozen Vegetables', 'Meal Prep', 'Fitness Food', 'Health Recipes', 'Controlled Diet', 'Quick Meals', 'Broccoli', 'Cauliflower', 'Convenient'], 28: ['Pork Balls', 'Taiwan Pork', 'Hot Pot Ingredients', 'Fresh Meat', 'Pork Leg', 'Meatballs', 'Pure Pork', 'Yummy Soup', 'Local Ingredients', 'Meat Density'], 46: ['Gourmet Fish Dish', 'Taiwan Sea Bass', 'Post-surgery Diet', 'Meal Prep', 'Sea Bass Recipe', 'Nutritious Mea

In [8]:
import json
import os
assert set(single_inference_results) == set(final.AVAILABLE_PRODUCT_IDS)
log_dir = 'results/ckpt-14-2'
os.makedirs(log_dir, exist_ok=True)
with open(os.path.join(log_dir, 'generation_config.json'), 'w', encoding='utf8') as fout:
    json.dump(generation_config.to_dict(), fout, indent=4)
with open(os.path.join(log_dir, 'raw_predict.json'), 'w', encoding='utf8') as fout:
    json.dump(single_inference_results, fout, ensure_ascii=False)
# single_inference_results

In [1]:
"""Parse the results here, or if the results parsed offline, load it here."""

enhanced_tags: dict[int, list[str]] = final.data.enhanced_tags_rev()
# keys are pids, values are tags
assert set(enhanced_tags) == set(final.AVAILABLE_PRODUCT_IDS)
"""Save the enhanced tags if the results satisfy !!!!"""

'Save the enhanced tags if the results satisfy !!!!'

In [7]:
assert set(enhanced_tags) == set(final.AVAILABLE_PRODUCT_IDS)
from typing import Literal
import pandas as pd
rows = []
query_fn: Literal['sum', 'union', 'intersection', 'last'] = 'union'
"""The query_fn(reduce function) is one of 'sum', 'union', 'intersection', and 'last'.
You can play around and see if the results differ."""

for split, merged_df in final.data.merged_df().items():
    merged_df = final.utils.next_product_expansion(merged_df)
    baseline_scores = final.utils.baseline_score(merged_df)
    metrics = final.ranking_metrics(baseline_scores, merged_df['y_true'])
    metrics['name'] = f'baseline/{split}'
    rows.append(metrics)

    metrics = final.TagsEvaluator(PRODUCT_RAW_TAGS).cal_ranking_metrics(merged_df)
    metrics['name'] = f'raw_tags/{split}'
    rows.append(metrics)

    metrics = final.TagsEvaluator(enhanced_tags).cal_ranking_metrics(merged_df)
    metrics['name'] = f'enhanced_tags/{split}'
    rows.append(metrics)

res_df = pd.DataFrame(rows).set_index('name')
print(res_df)

                         hr@5     hr@10     hr@20     mrr@5    mrr@10  \
name                                                                    
baseline/train       0.207021  0.229455  0.255376  0.116818  0.119864   
raw_tags/train       0.238521  0.280832  0.325817  0.151135  0.156829   
enhanced_tags/train  0.261188  0.315006  0.360804  0.157282  0.164489   
baseline/val         0.206980  0.232186  0.258362  0.121619  0.125120   
raw_tags/val         0.248667  0.292293  0.342220  0.155082  0.160960   
enhanced_tags/val    0.254968  0.307804  0.351915  0.153466  0.160651   
baseline/test        0.216379  0.246431  0.272727  0.118207  0.122326   
raw_tags/test        0.256198  0.302780  0.340346  0.162935  0.169362   
enhanced_tags/test   0.274230  0.343351  0.389932  0.159492  0.169051   

                       mrr@20    ndcg@5   ndcg@10   ndcg@20  
name                                                         
baseline/train       0.121639  0.139680  0.146988  0.153512  
raw_tags/t